In [2]:
import pandas as pd
import time

# Start timer
start_timer = time.time()

#Convert accepted_loans data to Panda DataFrame
accepted_df = pd.read_csv("/Users/abubakaral-faki/Documents/Data Project/MPV1/data/raw/accepted_2007_to_2018Q4.csv")

# End timer
end_timer = time.time()

/var/folders/yj/v54tzbn14gb70ts6s0j5s3xm0000gn/T/ipykernel_1690/674185456.py:8: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv("/Users/abubakaral-faki/Documents/Data Project/MPV1/data/raw/accepted_2007_to_2018Q4.csv")


In [3]:
print(f"Data took {end_timer - start_timer:.2f} seconds to load.")

Data took 18.75 seconds to load.


In [6]:
#create a copy of data frame to work with temporarily before applying changes to original 'accepted_df' DataFrame
accepted_df_copy = pd.DataFrame(accepted_df)

print(accepted_df_copy.head())

         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months   
3  66310712        NaN    35000.0      35000.0          35000.0   60 months   
4  68476807        NaN    10400.0      10400.0          10400.0   60 months   

   int_rate  installment grade sub_grade  ... hardship_payoff_balance_amount  \
0     13.99       123.03     C        C4  ...                            NaN   
1     11.99       820.28     C        C1  ...                            NaN   
2     10.78       432.66     B        B4  ...                            NaN   
3     14.85       829.90     C        C5  ...                            NaN   
4     22.45       289.91     F        F1  ...                            NaN   

  hardship_last_payment_amount disbursement_

In [8]:
accepted_df_copy['emp_title'].value_counts()

emp_title
Teacher                                38824
Manager                                34298
Owner                                  21977
Registered Nurse                       15867
Driver                                 14753
                                       ...  
Electronic System Engineer                 1
Substitute Teacher/Paraprofessional        1
eligiblity worker                          1
Escalated Customer Service Advisor         1
Current Operations Officer                 1
Name: count, Length: 512694, dtype: int64

### Create a CSV of each column and it's datatype for manual inspection

In [11]:
# Get data types of each variable
variable_data_types_df = accepted_df.dtypes.reset_index()

# Rename columns
variable_data_types_df.columns = ['Variables', 'Data Type']
print(variable_data_types_df)

#Save as csv
try:
    file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/variables_data_type.csv'
    
    variable_data_types_df.to_csv(file_path, index = False)
    print("Successfully saved variable_data_types_df as a csv.")

except Exception as e:
    print(e)

                 Variables Data Type
0                       id    object
1                member_id   float64
2                loan_amnt   float64
3              funded_amnt   float64
4          funded_amnt_inv   float64
..                     ...       ...
146      settlement_status    object
147        settlement_date    object
148      settlement_amount   float64
149  settlement_percentage   float64
150        settlement_term   float64

[151 rows x 2 columns]
Successfully saved variable_data_types_df as a csv.


## Convert Date columns to Date Data Types

In [14]:
# Convert Date_columns to Date Data types
date_columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d',
                'last_credit_pull_d', 'sec_app_earliest_cr_line', 'hardship_start_date',
                'hardship_end_date', 'payment_plan_start_date', 'debt_settlement_flag_date', 'settlement_date']

print(len(date_columns))

11


In [16]:
# Check format of date before converting to date objects

# We use dropna to drop all missing values in a columns because sometimes the first few rows have missing values

accepted_df.settlement_date.dropna().head() # Date format "%b-$Y"

68     Sep-2017
99     Nov-2017
105    Jan-2018
143    May-2018
144    Sep-2017
Name: settlement_date, dtype: object

## Check if all dates in each column are the same format

In [19]:
# Function to check format of date columns
def validate_date_format(series, date_format = '%b-%Y'):
    try:
        pd.to_datetime(series, format = date_format, errors = 'raise')
        return True
        
    except ValueError:
        return False
    
#Check is all columns have same format '%b-%Y' e.g. Sep-2017
validation_results = {col: validate_date_format(accepted_df[col]) for col in date_columns}

# Display results of validate_results
for column, is_valid in validation_results.items():
    print(f"Column '{column}' follows format '%b-%Y': {is_valid}")

Column 'issue_d' follows format '%b-%Y': True
Column 'earliest_cr_line' follows format '%b-%Y': True
Column 'last_pymnt_d' follows format '%b-%Y': True
Column 'next_pymnt_d' follows format '%b-%Y': True
Column 'last_credit_pull_d' follows format '%b-%Y': True
Column 'sec_app_earliest_cr_line' follows format '%b-%Y': True
Column 'hardship_start_date' follows format '%b-%Y': True
Column 'hardship_end_date' follows format '%b-%Y': True
Column 'payment_plan_start_date' follows format '%b-%Y': True
Column 'debt_settlement_flag_date' follows format '%b-%Y': True
Column 'settlement_date' follows format '%b-%Y': True


## Convert all date_columns to datetime64

In [22]:
# Specify the correct format: %b for abbreviated month name, %Y for 4-digit year
date_format = "%b-%Y"

# Convert date columns efficiently using the specified format
accepted_df_copy[date_columns] = accepted_df_copy[date_columns].apply(lambda col: pd.to_datetime(col,format =  date_format, errors = 'coerce'))

# Check if dates were formatted correctly
print(accepted_df_copy.issue_d.head())

0   2015-12-01
1   2015-12-01
2   2015-12-01
3   2015-12-01
4   2015-12-01
Name: issue_d, dtype: datetime64[ns]


In [23]:
# Check if all date columns are datetime datetypes
accepted_df_copy[date_columns].dtypes

issue_d                      datetime64[ns]
earliest_cr_line             datetime64[ns]
last_pymnt_d                 datetime64[ns]
next_pymnt_d                 datetime64[ns]
last_credit_pull_d           datetime64[ns]
sec_app_earliest_cr_line     datetime64[ns]
hardship_start_date          datetime64[ns]
hardship_end_date            datetime64[ns]
payment_plan_start_date      datetime64[ns]
debt_settlement_flag_date    datetime64[ns]
settlement_date              datetime64[ns]
dtype: object

In [26]:
# Check if all date columns are datetime datetypes
assert all(accepted_df_copy[col].dtype == 'datetime64[ns]' for col in date_columns)

## Convert categorical columns to 'category' type

In [29]:
# Convert Categorical columns to category

categorical_columns = ['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
                       'verification_status', 'loan_status', 'pymnt_plan', 'purpose'
                       ,'addr_state', 'initial_list_status', 'application_type',
                       'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status',
                       'hardship_loan_status', 'disbursement_method', 'debt_settlement_flag',
                       'settlement_status']

accepted_df_copy[categorical_columns] = accepted_df_copy[categorical_columns].apply(lambda col: col.astype('category'))

In [30]:
accepted_df_copy[categorical_columns].describe()

,term,grade,sub_grade,emp_length,home_ownership,verification_status,loan_status,pymnt_plan,purpose,addr_state,initial_list_status,application_type,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_loan_status,disbursement_method,debt_settlement_flag,settlement_status
count,2260668,2260668,2260668,2113761,2260668,2260668,2260668,2260668,2260668,2260668,2260668,2260668,2260668,10917,10917,10917,10917,2260668,2260668,34246
unique,2,7,35,11,6,3,9,2,14,51,2,2,2,1,9,3,5,2,2,3
top,36 months,B,C1,10+ years,MORTGAGE,Source Verified,Fully Paid,n,debt_consolidation,CA,w,Individual,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,Late (16-30 days),Cash,N,ACTIVE
freq,1609754,663557,145903,748005,1111450,886231,1076751,2260048,1277877,314533,1535467,2139958,2259836,10917,2965,7819,4770,2182546,2226422,14704


In [31]:
# Check if categorical columns are of categorical data type

accepted_df_copy[categorical_columns].dtypes

term                    category
grade                   category
sub_grade               category
emp_length              category
home_ownership          category
verification_status     category
loan_status             category
pymnt_plan              category
purpose                 category
addr_state              category
initial_list_status     category
application_type        category
hardship_flag           category
hardship_type           category
hardship_reason         category
hardship_status         category
hardship_loan_status    category
disbursement_method     category
debt_settlement_flag    category
settlement_status       category
dtype: object

In [35]:
#Check if all cetagorical columns are now of category type

assert all(accepted_df_copy[col].dtype == 'category' for col in categorical_columns)

In [37]:
# the title column will require some cleaning because the categories appear in a messy format so I am converting back to a strings

'''
['\tcredit_card', '\tdebt_consolidation', '\thouse', '\tother', ..., 'zxcvb', '~Life Reorganization~', 
'~Summer Fun~', 'îîMY FIRST CAR îî']

'''

accepted_df_copy['title']

0          Debt consolidation
1                    Business
2                         NaN
3          Debt consolidation
4              Major purchase
                  ...        
2260696                   NaN
2260697    Debt consolidation
2260698                   NaN
2260699                   NaN
2260700                   NaN
Name: title, Length: 2260701, dtype: object

# Convert 'id' column to int

In [40]:
#check if 'id' column has any missing values
print(accepted_df_copy['id'].describe())

#convert id column from objext(text) to int

'''This code below didn't work because there is a row that is a string representing this value;
 'Total amount funded in policy code 1: 6417608175'
'''
#accepted_df_copy['id'] = accepted_df_copy['id'].astype(int)

# Check if all entries are numbers
print(all(accepted_df_copy['id'].astype(str).str.isdigit()))

count      2260701
unique     2260701
top       68407277
freq             1
Name: id, dtype: int64
False


In [41]:
# identify all rows in 'id' column that are not integers

non_int_rows = accepted_df_copy[~accepted_df_copy['id'].apply(lambda x: str(x).isdigit())] #33 rows

print(len(non_int_rows['id']), 'non-int rows identified in id column')
print('\n')
print(non_int_rows['id'])

33 non-int rows identified in id column


421095     Total amount funded in policy code 1: 6417608175
421096     Total amount funded in policy code 2: 1944088810
528961     Total amount funded in policy code 1: 1741781700
528962      Total amount funded in policy code 2: 564202131
651664     Total amount funded in policy code 1: 1791201400
651665      Total amount funded in policy code 2: 651669342
749520     Total amount funded in policy code 1: 1443412975
749521      Total amount funded in policy code 2: 511988838
877716     Total amount funded in policy code 1: 2063142975
877717      Total amount funded in policy code 2: 823319310
983169     Total amount funded in policy code 1: 1538432075
983170      Total amount funded in policy code 2: 608903141
1117058    Total amount funded in policy code 1: 2087217200
1117059     Total amount funded in policy code 2: 662815446
1352689    Total amount funded in policy code 1: 3503840175
1352690     Total amount funded in policy code 2: 87365273

In [42]:
# check if all columns associated with the non_int_rows in 'id' column are empty

col_not_empty = non_int_rows.drop(columns=['id']).isna().all(axis=1)

if (col_not_empty.all() == True):
    print('All columns associated with non-int rows in \'id\' columns are empty')
else:
    print('All columns associated with non-int rows in \'id\' columns are\'nt empty')

All columns associated with non-int rows in 'id' columns are empty


In [44]:
# Drop all non-int rows in id column

print('accepted_df shape before removing rows:', accepted_df_copy.shape) #Get shape of accepted_df Dataframe before dropping rows

# Step 1 - Get index of non-int rows in 'id' column to drop
nonint_rows_todrop = non_int_rows.index

print(f"Indexes of Non-integer rows to drop : {nonint_rows_todrop}\n")
print(f"non-int rows to remove: {len(nonint_rows_todrop)}\n\n")

# Step 2 - remove non-int rows in 'id' column from accepted_df_copy
accepted_df_copy = accepted_df_copy.drop(index = nonint_rows_todrop)
print(len(nonint_rows_todrop), 'successfully dropped\n\n')

# Step 3 - Check accepted_df shape to confirm if 32 rows were dropped

print('accepted_df shape after removing rows:', accepted_df_copy.shape)


accepted_df shape before removing rows: (2260701, 151)
Indexes of Non-integer rows to drop : Index([ 421095,  421096,  528961,  528962,  651664,  651665,  749520,  749521,
        877716,  877717,  983169,  983170, 1117058, 1117059, 1352689, 1352690,
       1481103, 1481104, 1611877, 1611878, 1651665, 1654415, 1654416, 1751196,
       1751197, 1939379, 1939380, 2038501, 2038502, 2157151, 2157152, 2260699,
       2260700],
      dtype='int64')

non-int rows to remove: 33


33 successfully dropped


accepted_df shape after removing rows: (2260668, 151)


In [47]:
# Check if we still have non-int rows in 'id' column
accepted_df_copy[~accepted_df_copy['id'].apply(lambda row: str(row).isdigit())] #Should return an empty DF

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term


In [48]:
# Soooo now we can finally Convert 'id' column to integer
accepted_df_copy['id'] = accepted_df_copy['id'].astype(int)

In [49]:
accepted_df_copy['id'].info()

<class 'pandas.core.series.Series'>
Index: 2260668 entries, 0 to 2260698
Series name: id
Non-Null Count    Dtype
--------------    -----
2260668 non-null  int64
dtypes: int64(1)
memory usage: 34.5 MB


### Check for other variables of type 'object' so we can convert to the right type if necessary

In [51]:
#Get data types of all variables and convert it to a DataFrame
var_data_types = pd.DataFrame(accepted_df_copy.dtypes)

# Reset index of Dataframe
var_data_types = var_data_types.reset_index()

#define column names
var_data_types.columns = ['variable', 'type']

#Get variables that are type 'object'
object_vars = var_data_types[var_data_types['type'] == 'object']

print(object_vars)


                     variable    type
10                  emp_title  object
18                        url  object
19                       desc  object
21                      title  object
22                   zip_code  object
59  verification_status_joint  object


# Cleaning emp_title column

Convert emp_title to categorical variable and remove leading whitespaces from category names

In [56]:
# Convert emp_title to category or leave as oject
accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].astype('category')

#Check type of emp_title
accepted_df_copy['emp_title'].cat.categories

Index(['\tCFO', '\tMultimedia Supervisor', '\tSlot technician',
       '\tVP - Operations', ' ', ' \tASR II', ' \tAdv Mtr Proj Fld Rep',
       ' \tAuto Body Repair', ' \tDriver', ' \tEmployee Strategies Manager',
       ...
       'zueck transportation', 'zulily', '{Owner}Truck Driver',
       '| Principal Business Solution Architect|',
       'År.  Technical Illustrator', '​Associate Tech Support Analyst',
       '​Financial Analyst', '​License Compliance Investigator',
       '​Senior IT Field Support', '👨‍🍳 '],
      dtype='object', length=512694)

In [59]:
career_list = accepted_df_copy['emp_title'].value_counts().reset_index()

career_list

,emp_title,count
0,Teacher,38824
1,Manager,34298
2,Owner,21977
3,Registered Nurse,15867
4,Driver,14753
...,...,...
512689,IT Specialsit,1
512690,IT Specialpist,1
512691,IT Speciality,1
512692,IT Specialists,1


In [62]:
accepted_df_copy['emp_title'].isna().sum() # count number of Missing values

166969

In [64]:
#Export Categories to CSV file for manual check

file_path = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/emp_title_categories.csv'

categories = accepted_df_copy['emp_title'].cat.categories

pd.DataFrame({'job_titles': categories}).to_csv(file_path, index = False)

From a quick overview of the CSV file I saw:

1.) numbers as job_titles
2.) job titles containing emojis
3.) duplicate Job_titles with spelling or formating differences
4.) text like this 'ÄãFinancial Analyst'
5.) Leading white spaces represented by '\t'

In [67]:
# Convert 'emp_title' back to string/object in Panadas
accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].astype(str)

In [69]:
# Remove Leading White spaces and Normalize text
accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].str.strip().str.lower()

In [70]:
accepted_df_copy['emp_title'].value_counts() # Check Value counts after standardizing text

emp_title
nan                                         166969
teacher                                      48460
manager                                      45852
owner                                        33591
registered nurse                             23354
                                             ...  
greenhouse manager/research project coor         1
sr. qa tech                                      1
committee legislative assistant                  1
avp, client administration                       1
exec dir internal audit                          1
Name: count, Length: 411811, dtype: int64

## Remove numbers that are job titles

In [74]:
#all job_titles as numbers in emp_title
number_job_titles = [title for title in categories if str(title).isdigit() == True]

print(number_job_titles)

print('\nThere are ', len(number_job_titles), 'job titles as numbers')

['0132', '03', '0301', '0321', '04', '1', '10', '1052', '1099', '11', '112', '12', '121000', '13', '14', '1400', '14000', '15', '15000', '16', '17', '1811', '19', '2', '20', '21', '22', '25', '25312', '3', '30', '30005', '30304', '36000', '37', '38645', '4', '45000', '48000', '5', '553742017', '59106', '6', '6048', '7', '8', '9', '911', '9652651']

There are  49 job titles as numbers


In [76]:
#Check frequency of numeric Job_title to check if they are important

numeric_titles = accepted_df_copy[accepted_df_copy['emp_title'].str.isdigit()]

print(numeric_titles['emp_title'].value_counts())

emp_title
1            13
3            12
8             9
5             8
12            7
10            7
11            6
4             4
2             4
911           4
20            3
15            3
14            3
6             3
21            2
17            2
112           2
16            2
7             1
1052          1
0132          1
04            1
25312         1
30304         1
1811          1
9652651       1
30            1
553742017     1
121000        1
1400          1
36000         1
37            1
1099          1
30005         1
59106         1
03            1
38645         1
19            1
6153          1
15000         1
9             1
25            1
45000         1
48000         1
0321          1
22            1
14000         1
6048          1
0301          1
13            1
Name: count, dtype: int64


In [78]:
#Remove numeric titles expect for 911 and 112 as 911 and 112 are numbers for emmergency services
import numpy as np
import re
keep_titles = ['911', '112']


def remove_numeric_job_tiles(title):
    ''' This function takes a column from a Dataframe and Keeps Numerics Job titles like 911 and 112
    But repaces other numeric job_titles with NaN'''
    
    if pd.isna(title): #Handle Missing Values to avoid errors
        return np.nan
    
    #Covert title to string and strip
    title_str = str(title).strip().lower()
    
    # Check if record is '911' or '112'
    if title_str in keep_titles:
        return title_str
    
    # Check job titles that have numbers in them e.g. '2nd pressman'
    if title_str.isdigit() and title_str not in keep_titles:
        return False # Replace meaningless numeric titles with NaN
    
    return title #Keep non-numeric titles unchanged


# Apply remove_numeric_job_tiles function to accepted_df_copy dataframe
# accepted_df_copy = accepted_df_copy[accepted_df_copy['emp_title'].apply(remove_numeric_job_tiles)].index #Store indexes of

accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].apply(remove_numeric_job_tiles)

In [79]:
#Check if we only have 6 numeric titles  
# - Recall we had six '911' titles and two '112' titles

print(accepted_df_copy['emp_title'][accepted_df_copy['emp_title'].str.isdigit() == True])

16225      911
116912     112
755033     911
900893     911
1465253    112
1696755    911
Name: emp_title, dtype: object


In [81]:
accepted_df_copy.shape

(2260668, 151)

## Remove commas periods and emojis

In [94]:
import pandas as pd
import re

# Ensure emp_title is a string and handle NaNs
accepted_df_copy['emp_title'] = accepted_df_copy['emp_title'].astype(str).fillna('')

# Function to check if a string consists only of special characters (excluding emojis)
def is_special_chars_only(text):
    text = text.strip()  # Remove spaces
    return bool(re.fullmatch(r"^[^\w\s-]+$", text))  # Matches only special characters (no letters/numbers)

'''def is_special_chars_only(text):
    # Remove common emoji unicode ranges from being considered "special characters"
    return pd.NA if (re.fullmatch(r'^[^\w\s\U0001F300-\U0001F9FF]+$', text)) else text'''

# Apply filtering: Remove rows where emp_title consists only of special characters
special_characters = accepted_df_copy[accepted_df_copy['emp_title'].apply(is_special_chars_only)]

# Display the filtered DataFrame
special_characters[['annual_inc', 'emp_title']]


,annual_inc,emp_title
470285,40000.0,`
1588940,25000.0,👨‍🍳
1677396,36000.0,.


In [96]:
special_characters_index = special_characters['emp_title'].index
print(special_characters_index)

Index([470285, 1588940, 1677396], dtype='int64')


In [98]:
print(accepted_df_copy.columns.get_loc('emp_title'))

10


In [100]:
# Replace special characters
replace_chef_emoji = "👨‍🍳"

for item in special_characters_index:
    # Replace chef emoji(👨‍🍳) with 'chef'
    if accepted_df_copy.loc[item, 'emp_title'] == replace_chef_emoji:
        accepted_df_copy.loc[item, 'emp_title'] = 'chef'
    else:
        # Replace Special characters with empty string ""
        accepted_df_copy.loc[item, 'emp_title'] = ""

In [104]:
accepted_df_copy.loc[special_characters_index,'emp_title']

470285         
1588940    chef
1677396        
Name: emp_title, dtype: object

# Identify misspelt Job titles in 'emp_title' using Fuzzy Matching Algorithm

In [107]:
#Install require package to use fuzzy matching algorithm
!pip install thefuzz

In [109]:
# create a list of top 20 job titles in dataset to correct
titles_to_correct = list(pd.DataFrame(accepted_df_copy['emp_title'].value_counts(ascending = False)).reset_index()['emp_title'][:21])

#remove 'nan' string values
print(titles_to_correct)

['nan', 'teacher', 'manager', 'owner', 'registered nurse', 'supervisor', 'driver', 'sales', 'rn', 'office manager', 'project manager', 'general manager', 'truck driver', 'director', 'president', 'engineer', 'sales manager', 'operations manager', 'police officer', 'vice president', 'technician']


In [111]:
# remove office manager, manager, project manager, general manager, sales manager, operations manager

remove_titles = {'manager', 'office manager', 'project manager', 'general manager', 'sales manager', 'operations manager'}
titles_to_correct = [job for job in titles_to_correct if job not in remove_titles]

print(titles_to_correct)

['nan', 'teacher', 'owner', 'registered nurse', 'supervisor', 'driver', 'sales', 'rn', 'truck driver', 'director', 'president', 'engineer', 'police officer', 'vice president', 'technician']


# Correct mispelled 'teacher' job titles

In [114]:
# Ensure job titles are lowercase and stripped of spaces
accepted_df_copy['c_emp_title'] = accepted_df_copy['emp_title']
accepted_df_copy['c_emp_title'] = accepted_df_copy['c_emp_title'].astype(str).str.lower().str.strip()

In [116]:
accepted_df_copy['emp_title'].value_counts()

emp_title
nan                                         166969
teacher                                      48460
manager                                      45852
owner                                        33591
registered nurse                             23354
                                             ...  
animal care tech 2                               1
food corriditor manger                           1
special worker                                   1
assistant vice president - mortgage comp         1
exec dir internal audit                          1
Name: count, Length: 411761, dtype: int64

In [118]:
from thefuzz import process
import pandas as pd
import time

# Get unique job titles
unique_titles = accepted_df_copy['c_emp_title'].dropna().unique()

# Start timer
start_timer = time.time()

# Use `process.extract()` to find closest matches to "teacher"
teachers = process.extract('teacher', unique_titles, limit=len(unique_titles))

# Filter matches to keep only those with a similarity score of 80 or higher
teachers_filtered = [(job, score) for job, score in teachers if score >= 91]

# Convert to DataFrame
df_teachers = pd.DataFrame(teachers_filtered, columns=['Job Title', 'Similarity Score'])

# Sort by similarity score in descending order
df_teachers = df_teachers.sort_values(by='Similarity Score', ascending=False)

# Create a mapping dictionary
teacher_map = {job: 'teacher' for job, score in teachers_filtered}
teachers_dict = pd.DataFrame(list(teacher_map.items()), columns = ['original job title', 'Mapped Job Title'])
teachers_dict.to_csv('/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/teachers_map.csv', index = False)

# Efficiently update values using `.map()`
#accepted_df_copy['c_emp_title'] = accepted_df_copy['c_emp_title'].map(teacher_map).fillna(accepted_df_copy['c_emp_title'])
accepted_df_copy.loc[accepted_df_copy['c_emp_title'].isin(teacher_map.keys()), 'c_emp_title'] = 'teacher'

# End timer
end_timer = time.time()

# Save results to CSV file
file_location = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/teachers_similarity_scores.csv'
df_teachers.to_csv(file_location, index=False)

# Check updated value counts
print(accepted_df_copy['c_emp_title'].value_counts()[:20])

print(f"Exceution time: {end_timer - start_timer:.2f} seconds.")

c_emp_title
nan                   166969
teacher                48829
manager                45852
owner                  33591
registered nurse       23354
supervisor             22306
driver                 22267
sales                  18984
rn                     17196
office manager         14231
project manager        13842
general manager        13317
truck driver           12797
director               10595
president               9826
engineer                8978
sales manager           8532
operations manager      8183
police officer          7675
vice president          7625
Name: count, dtype: int64
Exceution time: 1.09 seconds.


# Correct mispelled 'sales' job titles

In [121]:
from thefuzz import process
import pandas as pd
import time

# Get unique job titles
unique_titles = accepted_df_copy['c_emp_title'].dropna().unique()

# Start timer
start_timer = time.time()


# Use `process.extract()` to find closest matches to "sales"
sales = process.extract('sales', unique_titles, limit=len(unique_titles))

# Filter matches to keep only those with a similarity score of 80 or higher
sales_filtered = [(job, score) for job, score in sales if score > 90]

# Convert to DataFrame
sales_df = pd.DataFrame(sales_filtered, columns=['Job Title', 'Similarity Score'])

# Sort by similarity score in descending order
sales_df = sales_df.sort_values(by='Similarity Score', ascending=False)

# Create a mapping dictionary
sales_map = {job: 'sales' for job, score in sales_filtered}

# Efficiently update values using `.map()`
#accepted_df_copy['c_emp_title'] = accepted_df_copy['c_emp_title'].map(teacher_map).fillna(accepted_df_copy['c_emp_title'])
accepted_df_copy.loc[accepted_df_copy['c_emp_title'].isin(sales_map.keys()), 'c_emp_title'] = 'sales'

# End timer
end_timer = time.time()

# Save results to CSV file
file_location = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/job_title_matches/teachers_similarity_scores.csv'
sales_df.to_csv(file_location, index=False)

# Check updated value counts
print(accepted_df_copy['c_emp_title'].value_counts()[:20])

print(f"Exceution time: {end_timer - start_timer:.2f} seconds.")

c_emp_title
nan                   166969
teacher                48829
manager                45852
owner                  33591
registered nurse       23354
supervisor             22306
driver                 22267
sales                  19003
rn                     17196
office manager         14231
project manager        13842
general manager        13317
truck driver           12797
director               10595
president               9826
engineer                8978
sales manager           8532
operations manager      8183
police officer          7675
vice president          7625
Name: count, dtype: int64
Exceution time: 0.97 seconds.


# Correct mispelled 'owner' job titles

In [123]:
from thefuzz import process
import pandas as pd
import time

# Get unique job titles
unique_titles = accepted_df_copy['c_emp_title'].dropna().unique()

# Start timer
start_timer = time.time()


# Use `process.extract()` to find closest matches to "owner"
owner = process.extract('owner', unique_titles, limit=len(unique_titles))

# Filter matches to keep only those with a similarity score of 91 or higher
owner_filtered = [(job, score) for job, score in owner if score >= 91]

# Convert to DataFrame
owner_df = pd.DataFrame(owner_filtered, columns=['Job Title', 'Similarity Score'])

# Sort by similarity score in descending order
owner_df = owner_df.sort_values(by='Similarity Score', ascending = False)

# Create a mapping dictionary
owner_map = {job: 'sales' for job, score in owner_filtered}

# Efficiently update values using `.map()`
#accepted_df_copy['c_emp_title'] = accepted_df_copy['c_emp_title'].map(teacher_map).fillna(accepted_df_copy['c_emp_title'])
accepted_df_copy.loc[accepted_df_copy['c_emp_title'].isin(owner_map.keys()), 'c_emp_title'] = 'owner'

# End timer
end_timer = time.time()

# Save results to CSV file
file_location = '/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/job_title_matches/owner_similarity_scores.csv'
sales_df.to_csv(file_location, index=False)

# Check updated value counts
print(accepted_df_copy['c_emp_title'].value_counts()[:20])

print(f"Exceution time: {end_timer - start_timer:.2f} seconds.")

c_emp_title
nan                   166969
teacher                48829
manager                45852
owner                  33719
registered nurse       23354
supervisor             22306
driver                 22267
sales                  19003
rn                     17196
office manager         14231
project manager        13842
general manager        13317
truck driver           12797
director               10595
president               9826
engineer                8978
sales manager           8532
operations manager      8183
police officer          7675
vice president          7625
Name: count, dtype: int64
Exceution time: 0.99 seconds.


# Drop 'url' column it if not needed

In [127]:
#Inspect url Column
pd.set_option('display.max_colwidth', None) #make sure pandas doesn't truncate the url column


accepted_df_copy[['id', 'url']].head(5)

,id,url
0,68407277,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277
1,68355089,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089
2,68341763,https://lendingclub.com/browse/loanDetail.action?loan_id=68341763
3,66310712,https://lendingclub.com/browse/loanDetail.action?loan_id=66310712
4,68476807,https://lendingclub.com/browse/loanDetail.action?loan_id=68476807


Seems like the 'url' gets info for each specific 'id'. 

Let's check if the id from the id column matches the id in
in the url

In [130]:
# Extract the loan_id from the url using a regular expression
extracted_id = accepted_df_copy['url'].str.extract(r'loan_id=(\d+)')

In [131]:
#Check if the id was extracted correctly from the url
print(extracted_id.head())

# Check type of extracted_id
print(type(extracted_id[0]))

          0
0  68407277
1  68355089
2  68341763
3  66310712
4  68476807
<class 'pandas.core.series.Series'>


#### Convert extracted_id to same datatype as 'id'


In [135]:
#Check dtype of id column
print('Type of \'id\' column is', accepted_df_copy['id'].dtype)

#Change dtype of extracted_id columns to 'int'
extracted_id[0] = extracted_id[0].astype('int')

#Check dtype of extracted_id column
print('Type of \'extracted_id\' is', extracted_id[0].dtype)

Type of 'id' column is int64
Type of 'extracted_id' is int64


In [137]:
#Check if all id's between 'extracted_id' column and 'id' column match

all_match = (accepted_df_copy['id'] == extracted_id[0]).all()

print(all_match)

True


#### We can drop url column since there no mismatches

In [140]:
# Shape before dropping 'url'
print('Shape before drop: ', accepted_df_copy.shape)

Shape before drop:  (2260668, 152)


In [142]:
# Drop 'url' column
accepted_df_copy = accepted_df_copy.drop('url', axis = 1)

In [144]:
# Shape after dropping 'url'
print('Shape after drop: ', accepted_df_copy.shape)

Shape after drop:  (2260668, 151)


### Inspect 'desc' column and change 'desc' column to the right datatype if needed

'desc' contains information about the loan description wriiten by the borrower.

In [148]:
print(accepted_df_copy['desc'].head())

print(accepted_df_copy['desc'].dtype)


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: desc, dtype: object
object


#### Notes from code above
From checking the data type of 'desc' and the first few rows it seems this column might be empty.

'desc' contains information about the loan description form the borrower.

We will rename this column to make it more intuitive to understand.

In [151]:
#Check if all values in the 'desc' column are all NaN

print(accepted_df_copy['desc'].info())

<class 'pandas.core.series.Series'>
Index: 2260668 entries, 0 to 2260698
Series name: desc
Non-Null Count   Dtype 
--------------   ----- 
126065 non-null  object
dtypes: object(1)
memory usage: 99.0+ MB
None


In [153]:
#count nuber of Null Values in 'desc' column

print(accepted_df_copy['desc'].isna().sum())

2134603


In [155]:
#Change name of column 'desc' to 'loan_purpose'
accepted_df_copy = accepted_df_copy.rename(columns = {'desc': 'loan_purpose'}) 

In [156]:
#Check if name change happened

accepted_df_copy['loan_purpose'].info()

<class 'pandas.core.series.Series'>
Index: 2260668 entries, 0 to 2260698
Series name: loan_purpose
Non-Null Count   Dtype 
--------------   ----- 
126065 non-null  object
dtypes: object(1)
memory usage: 99.0+ MB


In [157]:
#Check for missing values in 'desc'/'loan_purpose'
print(accepted_df_copy['loan_purpose'].isna().sum())

2134603


In [158]:
accepted_df_copy['loan_purpose'].unique()

array([nan,
       'We knew that using our credit cards to finance an adoption would squeeze us, but then medical and other unexpected expenses made the situation almost impossible. We are a stable family in a stable community. We just need to break a cycle of debt that is getting worse.',
       "I had a bad year two years ago, with some late and missed payments. I'm doing much better now, but I've got fees and some higher interest bits that have added up on top of the other stuff, and it's a little crazy. I'm hoping doing it thru Lending Club will make it easier - and cheaper - to pay off.",
       ...,
       'This loan will be used solely to consolidate credit card debts accrued while wife was/is unemployed.',
       'I have recently purchased and built a new home that I have always dreamed of having.  I would like to complete the project by putting a hottub in my backyard, however; I am not happy with the rate I  have been offered from GE to finance the spa.  I am paying cash for 

# Drop member_id columns

In [164]:
# Drop member_id column 
# All values for memeber_id are missing

accepted_df_copy.drop('member_id', axis = 1, inplace = True)

print('member_id successfuly dropped')

member_id successfuly dropped


## Export version 1 of dataframe

In [167]:
# Save a Version of the dataset with all columns cleaned except 'title' and 'employee'
file_path = "/Users/abubakaral-faki/Documents/Data Project/MPV1/data/interim/v1-accepted.csv"

accepted_df_copy.to_csv(file_path, index = False)

In [168]:
columns_dtypes = accepted_df_copy[accepted_df_copy.columns].dtypes
print(columns_dtypes.shape)
print(columns_dtypes)

(150,)
id                                int64
loan_amnt                       float64
funded_amnt                     float64
funded_amnt_inv                 float64
term                           category
                              ...      
settlement_date          datetime64[ns]
settlement_amount               float64
settlement_percentage           float64
settlement_term                 float64
c_emp_title                      object
Length: 150, dtype: object


In [169]:
columns_dtypes_df = columns_dtypes.reset_index()

columns_dtypes_df.columns = ['column', 'dtype']

columns_dtypes_df.head()

,column,dtype
0,id,int64
1,loan_amnt,float64
2,funded_amnt,float64
3,funded_amnt_inv,float64
4,term,category


In [170]:
file_path = "/Users/abubakaral-faki/Documents/Data Project/MPV1/temp_files/column_dtypes.csv"
columns_dtypes_df.to_csv(file_path, index = False)